In [5]:
import pyodbc
import pandas as pd
#from boxsdk import JWTAuth, Client
from datetime import datetime, timezone
import os
import logging
from logging.handlers import TimedRotatingFileHandler
import time
#import gspread
#from oauth2client.service_account import ServiceAccountCredentials
import csv
conn_str = (
    r'Driver={ODBC Driver 17 for SQL Server};'
    r'Server=dca1-up-dls;'
    r'Database=IACOE;'
    r'Trusted_Connection=yes;'
    )

cnxn = pyodbc.connect(conn_str)

cursor = cnxn.cursor()

query = """SELECT *
FROM (SELECT

    [Agent CAT Name],

    COUNT(CASE WHEN [Agent CAT Name] IS NOT NULL  THEN [Agent CAT Name] END) AS total_volume,

     COUNT(CASE WHEN [Agent CAT Name] IS NOT NULL  AND [Accuracy] = 1 THEN [Agent CAT Name] END) AS predicted_true_count
	 
FROM TaxML_Validation_Data where [Agent CAT Name] is not null

GROUP BY [Agent CAT Name]) AS e

ORDER BY e.predicted_true_count DESC"""

data = pd.read_sql(query,cnxn)

Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 17 for SQL Server' : file not found (0) (SQLDriverConnect)")

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Agent CAT Name        193 non-null    object
 1   total_volume          193 non-null    int64 
 2   predicted_true_count  193 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ KB


In [3]:
pd.set_option('display.max_rows', 500)

In [4]:
data['accuracy']=round((data['predicted_true_count']*100)/data['total_volume'],0)

In [11]:
data=data.sort_values(by=['accuracy','Agent CAT Name'],ascending = False)

In [17]:
data.to_csv('./data/accuracy_sheet.csv',index=False)

In [18]:
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

credentials = ServiceAccountCredentials.from_json_keyfile_name('./IACOE-GSuite-ServiceKey.json', scope)
client = gspread.authorize(credentials)

spreadsheet = client.open('Accuracy_INCA_prod')
# I modified below script.
sheetName = "Accuracy" # Please set the sheet name you want to append the CSV data.
#sheet_name = "Sheet3"  # Please put the sheet ID of the sheet you want to use.
csv_file = './data/accuracy_sheet.csv'  # Please put the file path of the CSV file you want to use.
worksheet = spreadsheet.worksheet(sheetName)
f = open(csv_file, "r")
values = [r for r in csv.reader(f)]
worksheet.update(values)

{'spreadsheetId': '1yWKliJsZPtqZc1v4aJ3j2FYwB7CuSBOnyepA-i9Q7Kw',
 'updatedRange': 'Accuracy!A1:D194',
 'updatedRows': 194,
 'updatedColumns': 4,
 'updatedCells': 776}